<a href="https://colab.research.google.com/github/bernaldiaz/bernaldiaz/blob/main/atmosferaBaumgartner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este programa es para calcular la s constantes atmosfericas en el caso del salto de Baumgartner


In [1]:
import numpy as np

In [2]:
# This should work up to and including 51,000 meters of altitude, using ISA values.

decimal_places = 5  # Amount of decimal places to give output to, adjustable.
gravity_const = 9.80665  # Gravity
R_const = 287.05  # Specific gas value for air

In [10]:
# Get and sanitize user input
altitude_str = input('Enter altitude in meters:')
try:
    altitude_flt = float(altitude_str)
    if altitude_flt < 0.0:
        raise ValueError
except ValueError:
    print('Altitude must be a positive float!')
    exit(0)
assert altitude_flt <= 51000, 'Altitude must be less than 51,000 meters!'


Enter altitude in meters:38969


In [11]:
# Different lapse rate values for different altitudes (in Kelvin/meter)
# This dict indicates the value from the previous layer up to the listed height
lapse_rate_values = {
    11000: -0.0065,
    20000: 0.000,
    32000: 0.001,
    47000: 0.0028,
    51000: 0.000
}

In [12]:
def calc_layer(t0, p0, rho0, delta_h, a):
    if a == 0.00:  # Isothermal layer
        ratio = np.e ** ((gravity_const / (-R_const * t0)) * delta_h)
        p1 = p0 * ratio
        rho1 = rho0 * ratio
        return t0, p1, rho1
    else:  # 'Normal' layer
        t1 = t0 + delta_h * a
        exponent = -(gravity_const / (R_const * a))
        t_ratio = (t1 / t0)
        p1 = t_ratio ** exponent * p0
        rho1 = t_ratio ** (exponent - 1) * rho0
        return t1, p1, rho1

In [13]:
alt_changing = altitude_flt  # Keep track of the altitude remaining to calculate
prev_p = 101325.0  # Pressure at sea level for ISA (Pa)
prev_rho = 1.2250  # Density at sea level for ISA (kg/m3)
prev_t = 288.15  # Temperature at sea level for ISA (K)
prev_layer_height = 0.0  # Sea level (m)

In [14]:
for layer_height, lapse_rate in lapse_rate_values.items():
    if alt_changing <= layer_height - prev_layer_height:
        # Alt changing is now the delta height, as it is the final 'layer' left
        final_t, final_p, final_rho = calc_layer(prev_t, prev_p, prev_rho, alt_changing, lapse_rate)
        print(f'Temperature at {altitude_flt}: {round(final_t, decimal_places)}K\n'
              f'Pressure at {altitude_flt}: {round(final_p, decimal_places)}Pa\n'
              f'Density at {altitude_flt}: {round(final_rho, decimal_places)}kg/m3')
        break
    else:
        dh = layer_height - prev_layer_height  # Compute delta height/thickness of layer
        alt_changing -= dh  # Calc alt changing, keeps track of how much altitude is left to calculate
        prev_t, prev_p, prev_rho = calc_layer(prev_t, prev_p, prev_rho, dh, lapse_rate)
        prev_layer_height = layer_height

Temperature at 38969.0: 248.1632K
Pressure at 38969.0: 319.56146Pa
Density at 38969.0: 0.00449kg/m3
